импортим

In [3]:
import torch 
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

если есть гпу

In [4]:
# если есть, то юзаем, если нет то цпу
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

параметры

In [5]:
#глобальные параметры
num_epochs = 5
num_classes = 10
batch_size = 100
learning_rate = 0.001

МНИСТ

In [6]:
# грузим мнист

train_dataset = torchvision.datasets.MNIST(root='../../data/',
                                           train=True, 
                                           transform=transforms.ToTensor(),
                                           download=True)

test_dataset = torchvision.datasets.MNIST(root='../../data/',
                                          train=False, 
                                          transform=transforms.ToTensor())

train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size, 
                                          shuffle=False)

Processing...
Done!


простая сетка

In [7]:
# сетка

class ConvNet(nn.Module):
    def __init__(self, num_classes=10):
        super(ConvNet, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.fc = nn.Linear(7*7*32, num_classes)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc(out)
        return out

model = ConvNet(num_classes).to(device)

In [9]:
# cross-entropy 
criterion = nn.CrossEntropyLoss()
# Adam
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)


In [10]:
total_step = len(train_loader)

In [11]:
total_step

600

In [12]:
for epoch in range(num_epochs): # если ЦПУ, то наверное 2-3 хватит, иначе ждать долго будем
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        
        
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

Epoch [1/5], Step [100/600], Loss: 0.1917
Epoch [1/5], Step [200/600], Loss: 0.1668
Epoch [1/5], Step [300/600], Loss: 0.0978
Epoch [1/5], Step [400/600], Loss: 0.0549
Epoch [1/5], Step [500/600], Loss: 0.1575
Epoch [1/5], Step [600/600], Loss: 0.1510
Epoch [2/5], Step [100/600], Loss: 0.0309
Epoch [2/5], Step [200/600], Loss: 0.0684
Epoch [2/5], Step [300/600], Loss: 0.0326
Epoch [2/5], Step [400/600], Loss: 0.0512
Epoch [2/5], Step [500/600], Loss: 0.0651
Epoch [2/5], Step [600/600], Loss: 0.0360
Epoch [3/5], Step [100/600], Loss: 0.0278
Epoch [3/5], Step [200/600], Loss: 0.0597
Epoch [3/5], Step [300/600], Loss: 0.0144
Epoch [3/5], Step [400/600], Loss: 0.0337
Epoch [3/5], Step [500/600], Loss: 0.0262
Epoch [3/5], Step [600/600], Loss: 0.0588
Epoch [4/5], Step [100/600], Loss: 0.0433
Epoch [4/5], Step [200/600], Loss: 0.0647
Epoch [4/5], Step [300/600], Loss: 0.0191
Epoch [4/5], Step [400/600], Loss: 0.0592
Epoch [4/5], Step [500/600], Loss: 0.0054
Epoch [4/5], Step [600/600], Loss:

In [13]:
model.eval()

ConvNet(
  (layer1): Sequential(
    (0): Conv2d(1, 16, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (0): Conv2d(16, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc): Linear(in_features=1568, out_features=10, bias=True)
)

In [14]:
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('accuracy: {} %'.format(100 * correct / total))

accuracy: 98.91 %
